## Importar librerías


In [1]:
import pandas as pd
import numpy as np
import os
import data_util as util

## Cargar datos AEMET (json)

In [2]:
ruta = '../data/metereologia/ano_2022'
ruta_nuevoCSV = '../data/auxiliar/meteo_2022.csv'
df_meteo = pd.read_json(ruta)
df_meteo.head(2)

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,presMax,horaPresMax,presMin,horaPresMin,dir,velmedia,racha,horaracha
0,2022-01-01,3195,"MADRID, RETIRO",MADRID,667,"11,1","0,0","5,5",07:10,"16,7",14:50,"951,5",11,"949,3",14,NaN,NaN,NaN,NaN
1,2022-01-02,3195,"MADRID, RETIRO",MADRID,667,"9,8","0,0","5,1",07:40,"14,5",15:00,"952,2",10,"949,5",16,NaN,NaN,NaN,NaN


## Seleccionar columnas

Nos quedamos con las columnas que nos interesan para el análisis

In [3]:
df_meteo.columns

Index(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec',
       'tmin', 'horatmin', 'tmax', 'horatmax', 'presMax', 'horaPresMax',
       'presMin', 'horaPresMin', 'dir', 'velmedia', 'racha', 'horaracha'],
      dtype='object')

In [4]:
df_meteo = df_meteo[['fecha', 'tmed', 'prec', 'dir', 'velmedia']]
df_meteo.head(2)

,fecha,tmed,prec,dir,velmedia
0,2022-01-01,"11,1","0,0",NaN,NaN
1,2022-01-02,"9,8","0,0",NaN,NaN


In [5]:
df_meteo.shape

(364, 5)

## Renombrar columnas

In [6]:
df_meteo = df_meteo.rename(columns={'fecha': 'FECHA', 'tmed': 'TEMPERATURA', 'prec': 'PRECIPITACION', 'dir': 'DIR_VIENTO', 'velmedia': 'VEL_VIENTO'})
df_meteo.head(2)

,FECHA,TEMPERATURA,PRECIPITACION,DIR_VIENTO,VEL_VIENTO
0,2022-01-01,"11,1","0,0",NaN,NaN
1,2022-01-02,"9,8","0,0",NaN,NaN


## Transformar columna FECHA

Transfomar la columna FECHA a formato datatime, para obtener datos horarios

In [7]:
df_meteo['FECHA'] = pd.to_datetime(df_meteo['FECHA'])
df_meteo['HORA'] = df_meteo['FECHA'].dt.hour
df_meteo.set_index('FECHA', inplace=True)
date_range = pd.date_range(start=df_meteo.index.min(), end=df_meteo.index.max(), freq='H')
df_meteo = df_meteo.reindex(date_range)
df_meteo.sort_index(inplace=True)
df_meteo.reset_index(inplace=True)
df_meteo.rename(columns={'index': 'FECHA'}, inplace=True)
df_meteo.head(2)

,FECHA,TEMPERATURA,PRECIPITACION,DIR_VIENTO,VEL_VIENTO,HORA
0,2022-01-01 00:00:00,"11,1","0,0",NaN,NaN,0.0
1,2022-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN


In [8]:
df_meteo.shape

(8737, 6)

## Rellenar los valores nulos

El método escogido es el último valor válido anterior, para que los datos horarios se rellenen con los datos que correponden a cada día

In [9]:
# Otra alternativa mejor es llenarlos con último valor válido anterior:
df_meteo.fillna(method='ffill', inplace = True )
df_meteo.head(2)

,FECHA,TEMPERATURA,PRECIPITACION,DIR_VIENTO,VEL_VIENTO,HORA
0,2022-01-01 00:00:00,"11,1","0,0",NaN,NaN,0.0
1,2022-01-01 01:00:00,"11,1","0,0",NaN,NaN,0.0


## Transformar valores

Se transforman los valores para que tengan el formato correcto

In [10]:
# Reemplazar las comas por puntos en la columna
df_meteo['TEMPERATURA'] = df_meteo['TEMPERATURA'].str.replace(',', '.')
df_meteo['PRECIPITACION'] = df_meteo['PRECIPITACION'].str.replace(',', '.')
df_meteo['VEL_VIENTO'] = df_meteo['VEL_VIENTO'].str.replace(',', '.')

# Convertir las columnas a los tipos de datos adecuados
df_meteo['TEMPERATURA'] = df_meteo['TEMPERATURA'].astype(float)
df_meteo['PRECIPITACION'] = df_meteo['PRECIPITACION'].astype(float)
df_meteo['VEL_VIENTO'] = df_meteo['VEL_VIENTO'].astype(float)

df_meteo.head(2)

,FECHA,TEMPERATURA,PRECIPITACION,DIR_VIENTO,VEL_VIENTO,HORA
0,2022-01-01 00:00:00,11.1,0.0,NaN,NaN,0.0
1,2022-01-01 01:00:00,11.1,0.0,NaN,NaN,0.0


In [11]:
util.calc_missing(df_meteo)

DIR_VIENTO 2160 / 8737= 24.72 %
VEL_VIENTO 2160 / 8737= 24.72 %


## Salvar datos preprocesados

In [12]:
df_meteo.isnull().sum()

FECHA               0
TEMPERATURA         0
PRECIPITACION       0
DIR_VIENTO       2160
VEL_VIENTO       2160
HORA                0
dtype: int64

In [13]:

df_meteo.to_csv(ruta_nuevoCSV, index=False) # Salvado con todos los datos